Импортируем библиотеки, настраиваем pyspark и читаем данные из файлов

In [45]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, sum as spark_sum, countDistinct, max as spark_max
from pyspark.sql.functions import radians, cos, sin, sqrt, atan2
from geopy.distance import geodesic

# Создание сессии Spark
spark = SparkSession.builder \
    .appName("BikeAnalysis") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Читаем CSV-файлы
trips_df = spark.read.option("header", "true").csv("trips.csv")
stations_df = spark.read.option("header", "true").csv("stations.csv")

# Приводим нужные колонки к числовому типу (иначе они строки)
stations_df = stations_df.withColumn("lat", col("lat").cast("double")) \
                         .withColumn("long", col("long").cast("double"))


# Задание 1
Найти велосипед с максимальным временем пробега

In [47]:
# Преобразуем даты в метки времени и считаем длительность поездки в минутах
trips_with_duration = trips_df.withColumn(
    "start_timestamp", unix_timestamp(col("start_date"), "M/d/yyyy H:mm")
).withColumn(
    "end_timestamp", unix_timestamp(col("end_date"), "M/d/yyyy H:mm")
).withColumn(
    "duration_minutes", (col("end_timestamp") - col("start_timestamp")) / 60
)

# Группируем по bike_id и находим велосипед с максимальным временем пробега
bike_max = trips_with_duration.groupBy("bike_id").agg(
    spark_sum("duration_minutes").alias("total_minutes")
).orderBy(col("total_minutes").desc()).limit(1)

# Вывод результата
bike_max.show()

+-------+-------------+
|bike_id|total_minutes|
+-------+-------------+
|    535|     310262.0|
+-------+-------------+



# Задание 2
Найти наибольшее геодезическое расстояние между станциями

In [48]:
# Применяем геодезическое расстояние (формула Хаверсина)
stations_list = stations_df.select("id", "lat", "long").collect()
max_distance = 0
station_pair = None

for i in range(len(stations_list)):
    for j in range(i + 1, len(stations_list)):
        s1, s2 = stations_list[i], stations_list[j]
        dist = geodesic((s1["lat"], s1["long"]), (s2["lat"], s2["long"])).km
        if dist > max_distance:
            max_distance = dist
            station_pair = (s1["id"], s2["id"])

print(f"Наибольшее расстояние: {max_distance:.2f} км между станциями {station_pair[0]} и {station_pair[1]}")

Наибольшее расстояние: 69.92 км между станциями 16 и 60


# Задание 3
Найти путь велосипеда с максимальным временем пробега через станции

In [49]:
# Получаем bike_id с максимальным пробегом
bike_id_max = bike_max.collect()[0]["bike_id"]

# Отбираем поездки для этого велосипеда и сортируем по start_date
bike_trips = trips_with_duration.filter(col("bike_id") == bike_id_max) \
    .select("start_date", "start_station_name", "end_date", "end_station_name") \
    .orderBy("start_date")

# Выводим путь
bike_trips.show(truncate=False)

+---------------+---------------------------------------------+---------------+---------------------------------------------+
|start_date     |start_station_name                           |end_date       |end_station_name                             |
+---------------+---------------------------------------------+---------------+---------------------------------------------+
|1/1/2014 13:42 |Mechanics Plaza (Market at Battery)          |1/1/2014 14:36 |Embarcadero at Sansome                       |
|1/1/2014 18:51 |Embarcadero at Sansome                       |1/1/2014 19:13 |Market at 4th                                |
|1/1/2014 19:48 |Market at 4th                                |1/1/2014 20:01 |South Van Ness at Market                     |
|1/10/2014 20:13|Market at 10th                               |1/10/2014 20:17|Powell Street BART                           |
|1/10/2014 8:09 |Embarcadero at Folsom                        |1/10/2014 8:19 |San Francisco Caltrain (Townsend at 4th

# Задание 4
Найти количество велосипедов в системе

In [50]:
# Используем countDistinct для подсчета уникальных bike_id
bike_count = trips_df.select(countDistinct("bike_id").alias("unique_bikes"))

# Вывод результата
bike_count.show()

+------------+
|unique_bikes|
+------------+
|         700|
+------------+



# Задание 5
Найти пользователей потративших на поездки более 3 часов.

In [51]:
# Фильтруем строки без zip_code
filtered_users = trips_with_duration.filter(col("zip_code").isNotNull())

# Группируем данные по zip_code и считаем общее время поездок
user_time = filtered_users.groupBy("zip_code").agg(
    spark_sum("duration_minutes").alias("total_minutes")
)

# Фильтруем пользователей, потративших более 180 минут (3 часа) на поездки
active_users = user_time.filter(col("total_minutes") > 180)

# Вывод результата
active_users.show()


+--------+-------------+
|zip_code|total_minutes|
+--------+-------------+
|   94102|     318746.0|
|   95134|      12114.0|
|   84606|       1583.0|
|   80305|       3010.0|
|   60070|        483.0|
|   95519|        505.0|
|   43085|        194.0|
|   91910|        840.0|
|   77339|        230.0|
|   48063|        228.0|
|   85022|        209.0|
|    1090|        340.0|
|    2136|        266.0|
|   11722|        405.0|
|   95138|       2583.0|
|   94610|      60490.0|
|   94404|      59813.0|
|   80301|       2537.0|
|   91326|       1096.0|
|   90742|        183.0|
+--------+-------------+
only showing top 20 rows

